In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json

In [248]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617
CIRCOM_PRIME = 21888242871839275222246405745257275088548364400416034343698204186575808495617
MAX_POSITIVE = CIRCOM_PRIME // 2
MAX_NEGATIVE = MAX_POSITIVE + 1 # The most positive number
CIRCOM_NEGATIVE_1 = 21888242871839275222246405745257275088548364400416034343698204186575808495617 - 1
EXPONENT = 15

class SeparableConv2D(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2D, self).__init__()
        self.dw_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False)
        self.pw_conv =  nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)

    def forward(self, x):
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        return x

def from_circom(x):
    if type(x) != int:
        x = int(x)
    if x > MAX_POSITIVE: 
        return x - CIRCOM_PRIME
    return x
    
def to_circom(x):
    if type(x) != int:
        x = int(x)
    if x < 0:
        return x + CIRCOM_PRIME 
    return x

In [249]:
def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    out = np.zeros((outRows, outCols, nFilters))
    remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row, col, channel] += input[row*strides+x, col*strides+y, channel] * weights[x, y, channel]
                
                out[row][col][channel] += bias[channel]
                remainder[row][col][channel] = out[row][col][channel] % n
                out[row][col][channel] = out[row][col][channel] / n
                            
    return out, remainder
    
def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    out = np.zeros((outRows, outCols, nFilters))
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for k in range(nChannels):
                    out[row, col, filter] += input[row*strides, col*strides, k] * weights[k, filter]
                    
                out[row][col][filter] += bias[filter]
                out[row][col][filter] = out[row][col][filter] / n
                            
    return out

def SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1

    depthOut, rem = DepthwiseConv(nRows, nCols, nChannels, nDepthFilters, kernelSize, strides, n, input, depthWeights, depthBias)
    pointOut = PointwiseConv2d(outRows, outCols, nChannels, nPointFilters, strides, n, depthOut, pointWeights, pointBias)
    return pointOut

In [250]:
def DepthwiseConvInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[str(weights[i][j][k] % p) for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row][col][channel] += int(input[row*strides+x][col*strides+y][channel]) * int(weights[x][y][channel])
                        
                out[row][col][channel] += int(bias[channel])
                remainder[row][col][channel] = str(int(out[row][col][channel] % n))
                out[row][col][channel] = int((out[row][col][channel] // n))
                str_out[row][col][channel] = str(out[row][col][channel] % p)
                
    return Input, Weights, Bias, out, str_out, remainder
    
def PointwiseConv2dInt(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[str(weights[k][l] % p) for l in range(nFilters)] for k in range(nChannels)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for channel in range(nChannels):
                    out[row][col][filter] += int(input[row*strides][col*strides][channel]) * int(weights[channel][filter])
                            
                out[row][col][filter] += int(bias[filter])
                remainder[row][col][filter] = str(int(out[row][col][filter] % n))
                out[row][col][filter] = str(out[row][col][filter] // n % p)
    return Input, Weights, Bias, out, remainder

def SeparableConvInt(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1

    Input, DepthWeights, DepthBias, depthOut, depthStrOut, depthRemainder = DepthwiseConvInt(nRows, nCols, nChannels, nDepthFilters, kernelSize, strides, n, input, depthWeights, depthBias)
    test = [[[from_circom(int(depthStrOut[i][j][k])) for k in range(3)] for j in range(5)] for i in range(5)]
    assert(test == depthOut)
    pInput, PointWeights, PointBias, pointOut, pointRem = PointwiseConv2dInt(outRows, outCols, nChannels, nPointFilters, strides, n, depthOut, pointWeights, pointBias)
    
    return Input, DepthWeights, DepthBias, depthOut, depthStrOut, depthRemainder, PointWeights, PointBias, pointOut, pointRem

In [251]:
input = torch.randn((1, 3, 5, 5))
model = SeparableConv2D(3, 6)

In [252]:
weights = model.dw_conv.weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))

expected = model.dw_conv(input).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 1, padded, weights, bias)
expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

In [253]:
weights = model.pw_conv.weight.detach().numpy()
print(f"{weights.shape=}")
bias = torch.zeros(weights.shape[0]).numpy()

expected = model.pw_conv(input).detach().numpy()

# # Converting to H x W x C
padded = input.squeeze().numpy().transpose((1, 2, 0))
print(padded.shape)
weights = weights.transpose((2, 3, 1, 0)).squeeze()

actual = PointwiseConv2d(5, 5, 3, 6, 1, 1, padded, weights, bias)

expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

weights.shape=(6, 3, 1, 1)
(5, 5, 3)


In [254]:
depthWeights = model.dw_conv.weight.squeeze().detach().numpy()
depthBias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))
depthWeights = depthWeights.transpose((1, 2, 0))

pointWeights = model.pw_conv.weight.detach().numpy()
print(f"{weights.shape=}")
pointBias = torch.zeros(pointWeights.shape[0]).numpy()
pointWeights = pointWeights.transpose((2, 3, 1, 0)).squeeze()

expected = model(input).detach().numpy()
print(f"{expected.shape=}")
# pointInput = model.dw_conv(input)
# expected = model.pw_conv(pointInput).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))

print(pointBias.shape)
# SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
actual = SeparableConvImpl(7, 7, 3, 3, 6, 3, 1, 1, padded, depthWeights, pointWeights, depthBias, pointBias)
expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

weights.shape=(3, 6)
expected.shape=(1, 6, 5, 5)
(6,)


In [255]:
depthWeights = model.dw_conv.weight.squeeze().detach().numpy()
depthBias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))
depthWeights = depthWeights.transpose((1, 2, 0))

pointWeights = model.pw_conv.weight.detach().numpy()
print(f"{weights.shape=}")
pointBias = torch.zeros(pointWeights.shape[0]).numpy()
pointWeights = pointWeights.transpose((2, 3, 1, 0)).squeeze()

expected = model(input).detach().numpy()
print(f"{expected.shape=}")
# pointInput = model.dw_conv(input)
# expected = model.pw_conv(pointInput).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))

print(pointBias.shape)
# SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
actual = SeparableConvImpl(7, 7, 3, 3, 6, 3, 1, 1, padded, depthWeights, pointWeights, depthBias, pointBias)
expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual))

def SeparableConvTesting(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    assert(nDepthFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    depthOut = np.zeros((outRows, outCols, nDepthFilters))
    depthRemainder = np.zeros((outRows, outCols, nDepthFilters))

    # Depthwise convolution
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        # depthOut[row, col, channel] += input[row*strides+x, col*strides+y, channel] * depthWeights[x, y, channel]
                        depthOut[row][col][channel] += int(input[row*strides+x, col*strides+y, channel]) * int(depthWeights[x, y, channel])
                
                depthOut[row][col][channel] += depthBias[channel]
                depthRemainder[row][col][channel] = depthOut[row][col][channel] % n
                depthOut[row][col][channel] = depthOut[row][col][channel] // n
                
    # Pointwise convolution
    out = np.zeros((outRows, outCols, nPointFilters))
    remainder = np.zeros((outRows, outCols, nPointFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nPointFilters):
                for k in range(nDepthFilters):
                    out[row, col, filter] += depthOut[row*strides, col*strides, k] * pointWeights[k, filter]
                    
                out[row][col][filter] += pointBias[filter]
                out[row][col][filter] = out[row][col][filter] // n
                
    return out, remainder, depthOut, depthRemainder
    
depthWeights = model.dw_conv.weight.squeeze().detach().numpy()
depthBias = torch.zeros(weights.shape[0], dtype=int).numpy()
# input = torch.randn((1, 8, 32, 32))
depthWeights = depthWeights.transpose((1, 2, 0))

pointWeights = model.pw_conv.weight.detach().numpy()
print(f"{weights.shape=}")
pointBias = torch.zeros(pointWeights.shape[0]).numpy()
pointWeights = pointWeights.transpose((2, 3, 1, 0)).squeeze()

expected = model(input).detach().numpy()
print(f"{expected.shape=}")
# pointInput = model.dw_conv(input)
# expected = model.pw_conv(pointInput).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))

print(pointBias.shape)
# SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):

quantized_image = padded * 10**EXPONENT
quantized_depth_weights = depthWeights * 10**EXPONENT
quantized_point_weights = pointWeights * 10**EXPONENT

actual, _, depthOut, depthRemainder = SeparableConvTesting(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias, pointBias)
expected = expected.squeeze().transpose((1, 2, 0))

print(f"{actual[0][0][0]=}")
print(f"{expected[0][0][0]=}")

actual = actual / 10**EXPONENT
assertDepthOut = depthOut / 10**EXPONENT

dw_expected = model.dw_conv(input)
pw_expected = model.pw_conv(dw_expected).detach().numpy()
print(f"{dw_expected.shape=}")
dw_expected = dw_expected.detach().numpy().squeeze().transpose((1, 2, 0))
print(f"{dw_expected.shape=}")
pw_expected = pw_expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual))
assert(np.allclose(expected, pw_expected))
assert(np.allclose(assertDepthOut, dw_expected))

input_json_path = "depthwiseConv2D_input_test.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": quantized_image.round().astype(int).tolist(),
               "weights": quantized_depth_weights.round().astype(int).tolist(),
               "remainder": depthRemainder.round().astype(int).tolist(),
               "out": depthOut.round().astype(int).tolist(),
               "bias": depthBias.round().astype(int).tolist(),
              },
              input_file)


print(dw_expected[0][0][0])
print(depthOut.tolist()[0][0][0])
print(f"{to_circom(-1695193072.0)=}")
print(f"{to_circom(-1695193072)=}")

print(to_circom(-1695193072) == 21888242871839275222246405745257275088548364400416034343698204186574113302545)
to_circom(-1695193072 * 10**EXPONENT) - 21888242871839275222246405745257275088548364400416034343698034667268608495617
21888242871839275222246405745257275088548364400416034343698034667268608495617 - 21888242871839275222246405745257275088548364400416034343698204186574113302545
# ==from_circom(21888242871839275222246405745257275088548364400416034343698204186574113302545)=}")
# print(f"{from_circom(21888242871839275222246405745257275088548364400416034343681252255859160055228)=}")
# print(f"{from_circom(21888242871839275222246405745257275088548364400416034343698034667271960057857)=}")
# print(f"{from_circom(21888242871839275222246405745257275088548364400416034343681252255859160055228)=}")
# input_json_path = "separableConv2D_input.json"
# with open(input_json_path, "w") as input_file:
#     json.dump({"in": Input,
#                "depthWeights": DepthWeights,
#                "depthBias": DepthBias,
#                "depthRemainder": DepthRemainder,
#                "depthOut": DepthOut,
               
#                "pointWeights": PointWeights,
#                "pointBias": PointBias,
#                "pointRemainder": remainder,
#                "pointOut": out,
#               },
#               input_file)

weights.shape=(3, 6)
expected.shape=(1, 6, 5, 5)
(6,)
weights.shape=(3, 6)
expected.shape=(1, 6, 5, 5)
(6,)
actual[0][0][0]=17143474868210.0
expected[0][0][0]=0.017143477
dw_expected.shape=torch.Size([1, 3, 5, 5])
dw_expected.shape=(5, 5, 3)
-0.40900424
-409004260094254.0
to_circom(-1695193072.0)=21888242871839275222246405745257275088548364400416034343698204186574113302545
to_circom(-1695193072)=21888242871839275222246405745257275088548364400416034343698204186574113302545
True


-169519305504806928

In [256]:
padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
# weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_depth_weights = depthWeights * 10**EXPONENT
quantized_point_weights = pointWeights * 10**EXPONENT


    
    
# q_input, q_weights, bias, actual, str_actual, rem = DepthwiseConvInt(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), depthBias.astype(int))

# Input, DepthWeights, DepthBias, DepthOut, depthStrOut, depthRemainder, _, _, _, _ = SeparableConvTest(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias.astype(int), pointBias.astype(int))
Input, DepthWeights, DepthBias, depthOut, depthStrOut, DepthRem, PointWeights, PointBias, pointOut, pointRem = SeparableConvTest(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias.astype(int), pointBias.astype(int))
# Input, DepthWeights, DepthBias, depthOut, depthStrOut, depthRemainder, PointWeights, PointBias, remainder, out 

input_json_path = "separableConv2D_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": Input,
               "depthWeights": DepthWeights,
               "depthBias": DepthBias,
               "depthRemainder": DepthRem,
               "depthOut": depthStrOut,
               
               "pointWeights": PointWeights,
               "pointBias": PointBias,
               "pointRemainder": pointRem,
               "pointOut": pointOut,
              },
              input_file)


In [125]:
def SeparableConvTest(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    assert(nDepthFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    depthOut = np.zeros((outRows, outCols, nDepthFilters))
    depthRemainder = np.zeros((outRows, outCols, nDepthFilters))

    # Depthwise convolution
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        depthOut[row, col, channel] += input[row*strides+x, col*strides+y, channel] * depthWeights[x, y, channel]
                
                depthOut[row][col][channel] += depthBias[channel]
                depthRemainder[row][col][channel] = depthOut[row][col][channel] % n
                depthOut[row][col][channel] = depthOut[row][col][channel] // n
                            
    # Pointwise convolution
    out = np.zeros((outRows, outCols, nPointFilters))
    remainder = np.zeros((outRows, outCols, nPointFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nPointFilters):
                for k in range(nDepthFilters):
                    out[row, col, filter] += depthOut[row*strides, col*strides, k] * pointWeights[k, filter]
                    
                out[row][col][filter] += pointBias[filter]
                out[row][col][filter] = out[row][col][filter] // n
                
    return depthOut, depthRemainder, out
    
# quantized_image = padded * 10**EXPONENT
# quantized_depth_weights = depthWeights * 10**EXPONENT
# quantized_point_weights = pointWeights * 10**EXPONENT
    
# DepthOut, remainder, out = SeparableConvTest(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias.astype(int), pointBias.astype(int))
# # q_input, q_weights, bias, str_actual, actual, rem = DepthwiseConvInt(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), depthBias.astype(int))
# dw_expected = model.dw_conv(input)
# pw_expected = model.dw_conv(dw_expected)
# dw_expected = dw_expected.detach().numpy()
# pw_expected = pw_expected.detach().numpy()
# DepthOut = DepthOut / 10**EXPONENT
# dw_expected = dw_expected.squeeze().transpose((1, 2, 0))
# pw_expected = pw_expected.squeeze().transpose((1, 2, 0))
# print(DepthOut[0][0][0])
# print(dw_expected[0][0][0])
# print(out[0][0][0])
# print(pw_expected[0][0][0])

# assert(np.allclose(DepthOut, dw_expected))

depthWeights = model.dw_conv.weight.squeeze().detach().numpy()
depthBias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))
depthWeights = depthWeights.transpose((1, 2, 0))

pointWeights = model.pw_conv.weight.detach().numpy()
print(f"{weights.shape=}")
pointBias = torch.zeros(pointWeights.shape[0]).numpy()
pointWeights = pointWeights.transpose((2, 3, 1, 0)).squeeze()

print(f"{expected.shape=}")
# pointInput = model.dw_conv(input)
# expected = model.pw_conv(pointInput).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))

print(pointBias.shape)
# SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
depthOut, _, _ = SeparableConvTest(7, 7, 3, 3, 6, 3, 1, 10**EXPONENT, padded, depthWeights, pointWeights, depthBias, pointBias)
dw_expected = model.dw_conv(input).detach().numpy()
# expected = model(input).detach().numpy()
dw_expected = dw_expected.squeeze().transpose((1, 2, 0))

print(f"{depthOut[0][0][0]=}")
print(f"{dw_expected[0][0][0]=}")
assert(np.allclose(dw_expected, depthOut))

# quantized_image = padded * 10**EXPONENT
# quantized_depth_weights = depthWeights * 10**EXPONENT
# quantized_point_weights = pointWeights * 10**EXPONENT

# depthOut, _, _ = SeparableConvTest(7, 7, 3, 3, 6, 3, 1, 1, padded, quantized_image.round().astype(int), pointWeights, depthBias, pointBias)

# assert(np.allclose(expected, actual))

weights.shape=(3, 6)
expected.shape=(5, 5, 6)
(6,)
depthOut[0][0][0]=-1.0
dw_expected[0][0][0]=-0.1695193


AssertionError: 

In [60]:
# def SeparableConvTest(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
#     assert(nDepthFilters % nChannels == 0)
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
    
#     depthOut = np.zeros((outRows, outCols, nDepthFilters))
#     depthRemainder = np.zeros((outRows, outCols, nDepthFilters))

#     # Depthwise convolution
#     for row in range(outRows):
#         for col in range(outCols):
#             for channel in range(nChannels):
#                 for x in range(kernelSize):
#                     for y in range(kernelSize):
#                         depthOut[row, col, channel] += input[row*strides+x, col*strides+y, channel] * depthWeights[x, y, channel]
                
#                 depthOut[row][col][channel] += depthBias[channel]
#                 depthRemainder[row][col][channel] = depthOut[row][col][channel] % n
#                 depthOut[row][col][channel] = depthOut[row][col][channel] / n
                            
#     # Pointwise convolution
#     out = np.zeros((outRows, outCols, nPointFilters))
#     remainder = np.zeros((outRows, outCols, nPointFilters))
    
#     for row in range(outRows):
#         for col in range(outCols):
#             for filter in range(nPointFilters):
#                 for k in range(nDepthFilters):
#                     out[row, col, filter] += depthOut[row*strides, col*strides, k] * pointWeights[k, filter]
                    
#                 out[row][col][filter] += pointBias[filter]
#                 out[row][col][filter] = out[row][col][filter] / n
                
#     return out, remainder

# def SeparableConv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
def SeparableConvTest(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    DepthWeights = [[[str(depthWeights[i][j][l] % p) for l in range(nDepthFilters)] for j in range(kernelSize)] for i in range(kernelSize)]
    DepthBias = [str(int(bias[i] % p)) for i in range(nDepthFilters)]
    depthOut = [[[0 for _ in range(nDepthFilters)] for _ in range(outCols)] for _ in range(outRows)]
    depthStrOut = [[[0 for _ in range(nDepthFilters)] for _ in range(outCols)] for _ in range(outRows)]
    depthRemainder = [[[None for _ in range(nDepthFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        depthOut[row][col][channel] += input[row*strides+x, col*strides+y, channel] * depthWeights[x, y, channel]
                
                # remainder[row][col][filter] = str(out[row][col][filter] % n)
                # out[row][col][filter] = str(out[row][col][filter] // n % p)
                
                depthOut[row][col][channel] += depthBias[channel]
                depthRemainder[row][col][channel] = str(depthOut[row][col][channel] % n)
                depthOut[row][col][channel] = depthOut[row][col][channel] // n % p
                depthStrOut[row][col][channel] = str(depthOut[row][col][channel])
                
    PointWeights = [[[str(depthWeights[i][j][l] % p) for l in range(nDepthFilters)] for j in range(kernelSize)] for i in range(kernelSize)]
    PointBias = [str(bias[i] % p) for i in range(nDepthFilters)]
    PointOut = [[[0 for _ in range(nDepthFilters)] for _ in range(outCols)] for _ in range(outRows)]
    PointStrOut = [[[0 for _ in range(nDepthFilters)] for _ in range(outCols)] for _ in range(outRows)]
    PointRemainder = [[[None for _ in range(nDepthFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
    PointWeights = [[str(pointWeights[k][l] % p) for l in range(nPointFilters)] for k in range(nDepthFilters)]
    PointBias = [str(int(bias[i] % p)) for i in range(nPointFilters)]
    out = [[[0 for _ in range(nPointFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nPointFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nPointFilters):
                for channel in range(nDepthFilters):
                    # out[row][col][filter] += input[row*strides][col*strides][channel] * weights[channel][filter]
                        # print(type(depthOut[row*strides][col*strides][channel]))
                        # print(type(pointWeights[channel][filter].astype(int)))
                        # print(type(depthOut[row*strides][col*strides][channel] * pointWeights[channel][filter].astype(int)))
                        out[row][col][filter] += int(depthOut[row*strides][col*strides][channel]) * int(pointWeights[channel][filter].astype(int))
                            
                out[row][col][filter] += pointBias[filter]
                remainder[row][col][filter] = str(out[row][col][filter] % n)
                out[row][col][filter] = str(out[row][col][filter] // n % p)
                
    return Input, DepthWeights, DepthBias, depthRemainder, depthStrOut, PointWeights, PointBias, remainder, out 
    # return None, None


In [88]:
padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
# weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_depth_weights = depthWeights * 10**EXPONENT
quantized_point_weights = pointWeights * 10**EXPONENT

# q_input, q_weights, bias, actual, rem = DepthwiseConvInt(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_weights.round().astype(int), bias.astype(int))

Input, DepthWeights, DepthBias, DepthRemainder, DepthOut, PointWeights, PointBias, remainder, out  = \
    SeparableConvTest(7, 7, 3, 3, 6, 3, 1, 1, quantized_image.round().astype(int), quantized_depth_weights.round().astype(int), quantized_point_weights.round().astype(int), depthBias.astype(int), pointBias.astype(int))

# print(DepthOut)
input_json_path = "separableConv2D_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": Input,
               "depthWeights": DepthWeights,
               "depthBias": DepthBias,
               "depthRemainder": DepthRemainder,
               "depthOut": DepthOut,
               
               "pointWeights": PointWeights,
               "pointBias": PointBias,
               "pointRemainder": remainder,
               "pointOut": out,
              },
              input_file)

/tmp/ipykernel_46135/3291142876.py:42: RuntimeWarning: overflow encountered in scalar multiply
  out[row][col][filter] += input[row*strides][col*strides][channel] * weights[channel][filter]
/tmp/ipykernel_46135/3291142876.py:42: RuntimeWarning: overflow encountered in scalar add
  out[row][col][filter] += input[row*strides][col*strides][channel] * weights[channel][filter]


In [119]:
DepthBias

['0', '0', '0']